In [11]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
import sagemaker
from sagemaker.experiments.run import Run, load_run
import time
from datetime import datetime

sagemaker.__version__

'2.219.0'

In [18]:
verion = "v0.0.1"

In [12]:
session = sagemaker.Session()
sm = session.sagemaker_client

experiment_name = "team1-index-predictor-data-collection"
trial_suffix = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
trial_name = f"data-collection-{version}-{trial_suffix}"

bucket_name = "team1-index-predictor-bucket"
bucket_prefix = "data/raw"
data_s3_url = f"s3://{bucket_name}/{bucket_prefix}/"

processing_instance_type = "ml.t3.medium"
processing_instance_count = 1

In [13]:
with Run(
    experiment_name=experiment_name,
    run_name=trial_name,
    run_display_name=trial_name,
    sagemaker_session=session,
) as run:
    experiment_config = run.experiment_config

In [14]:
script_processor = ScriptProcessor(
    command=["python3"],
    image_uri="492215442770.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-scikit-learn:1.0-1-cpu-py3",
    role=sagemaker.get_execution_role(),
    instance_count=processing_instance_count,
    instance_type=processing_instance_type,
)

script_processor.run(
    code="../src/data/collector.py",
    inputs=[],
    outputs=[
        ProcessingOutput(
            source=f"/opt/ml/processing/{bucket_prefix}/data.csv",
            destination=data_s3_url,
            output_name="data",
        )
    ],
    arguments=[
        "--mode",
        "train-val-test",
    ],
    experiment_config=experiment_config,
)

INFO:sagemaker:Creating processing-job with name sagemaker-scikit-learn-2024-06-18-07-37-25-681


.........................................................................................Collecting yfinance
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 24.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 33.0 MB/s eta 0:00:00
   ━━━

In [17]:
!aws s3 ls s3://team1-index-predictor-bucket/data/raw/

2024-06-17 20:00:34       3589 sp500-inference.csv
2024-06-17 20:00:34     872565 sp500.csv
